In [ ]:
import json
import os
import cv2
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.callbacks import Callback,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
import numpy as np
print(os.listdir('../input'))
%matplotlib inline
#import visualkeras

In [ ]:
!pip install visualkeras
import visualkeras

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
train_df.head()

In [ ]:
train_df['diagnosis'].hist()

# DISPLAY

In [ ]:
def get_category(level):
    categories = ["Healthy","Mild", "Moderate", "Severe", "Proliferate"]
    return categories[level]

In [ ]:
def show_samples(df):
    fig=plt.figure(figsize=(20, 15))
    for i in range(12):
        image_path = df.loc[i,'id_code']
        image_diagnosis = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig.add_subplot(3, 4, i+1)
        plt.title(get_category(image_diagnosis))
        plt.axis('off')
        plt.imshow(img)
    plt.tight_layout()

show_samples(train_df)

In [ ]:
IMG_SIZE = 256
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0):
            return img 
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img
    
def preprocess_image(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)  
    return image

In [ ]:
def show_preprocessed(df):
    fig=plt.figure(figsize=(20, 15))
    for i in range(12):
        image_path = df.loc[i,'id_code']
        image_diagnosis = df.loc[i,'diagnosis']
        img = preprocess_image(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        fig.add_subplot(3, 4, i+1)
        plt.title(get_category(image_diagnosis))
        plt.axis('off')
        plt.imshow(img)
    plt.tight_layout()

show_preprocessed(train_df)

In [ ]:
N = train_df.shape[0]#3662
x_train = np.empty((N, IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
for i, image_id in enumerate(tqdm(train_df['id_code'])):#3662
    x_train[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/train_images/{image_id}.png')

In [ ]:
y_train = pd.get_dummies(train_df['diagnosis']).values

print(y_train.shape)
print(y_train[0], train_df['diagnosis'][0])

In [ ]:
y_train_multi_labels = np.empty(y_train.shape, dtype=y_train.dtype)#multi labels
y_train_multi_labels[:, 4] = y_train[:, 4]
#all lines for col 5 proliferative are the same as it is the last stage(initialization)
for i in range(3, -1, -1):
    y_train_multi_labels[:, i] = np.logical_or(y_train[:, i], y_train_multi_labels[:, i+1])
    #labels .. [:,3] = [:, 3] or [:, 4] i.e if 4 is there it means 3 is up (multi labeling)
    #i.e if it s Prolif its gonna be [1,1,1,1,1], if healthy [1,0,0,0,0] (multi label problem)

print(f"y_train: {y_train.sum(axis=0)}, example: {y_train[0]}")
print(f"Multilabel version: {y_train_multi_labels.sum(axis=0)}, example: {y_train_multi_labels[0]}")
#all proliferative are also severe but severe are not proliferative and so on

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train_multi_labels, test_size=0.20)
X_val = X_val/255# we normalize later the test vals through generator
print((X_train.shape, X_val.shape))

In [ ]:
with open('train_test.npy', 'wb') as f:
    np.save(f, X_train)
    np.save(f, X_val)
    np.save(f, Y_train)
    np.save(f, Y_val)


In [ ]:
from IPython.display import FileLink
FileLink(r'train_test.npy')

In [ ]:
#with open('train_test.npy', 'rb') as f:
   # X_train = np.load(f)
   # X_val = np.load(f)
   # Y_train = np.load(f)
   # Y_val = np.load(f)
X_train.shape

In [ ]:
train_df['diagnosis']=train_df['diagnosis'].astype(str)#for categorical diagnosis
def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.15,  # set range for random zoom
        # set mode for filling points outside the input boundaries
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,# randomly flip images
        rotation_range=360,
        rescale=1./255)

In [ ]:
batch_size = 16
data_generator = create_datagen().flow(X_train, Y_train, batch_size=batch_size)

# Kappa

In [ ]:
from sklearn.metrics import accuracy_score

class Metrics(Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data
        
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        

        X_val, y_val = self.validation_data[:2]
        y_val = y_val.sum(axis=1) - 1
        
        y_pred = self.model.predict(X_val) > 0.5
        y_pred = y_pred.astype(int).sum(axis=1) - 1
        
        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred, 
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"val_kappa: {_val_kappa:.4f}")
        print(f"Actual val_accuracy: {accuracy_score(y_val, y_pred)}")
        
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved... Saving model!")
            model.save_weights('model.h5')
            model_json = model.to_json()
            with open('model.json', "w") as json_file:
                json_file.write(model_json)
            json_file.close()

        return

In [ ]:
IMG_SIZE=256

In [ ]:
efficient = EfficientNetB5(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE,IMG_SIZE,3)
)

In [ ]:
def build_model():
    model = Sequential()
    model.add(efficient)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(5, activation='sigmoid'))
    
    
    model.compile(
        loss='binary_crossentropy',#to check why we use it in binary classif
        #our class contains this severity or n?
        optimizer=Adam(lr=0.00005),
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,show_layer_names=True)


In [ ]:
visualkeras.layered_view(model, legend=True)

In [ ]:
kappa_metrics = Metrics((X_val,Y_val))
stopping=EarlyStopping(monitor='val_loss',patience=5, min_delta=0.005)
history = model.fit_generator(
    data_generator,
    steps_per_epoch=X_train.shape[0] // batch_size,
    validation_data=(X_val,Y_val),
    epochs=12,
    callbacks=[stopping,kappa_metrics])

In [ ]:
#json_file = open('./model.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
#loaded_model.load_weights("./model.h5")
model.load_weights('model.h5')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
val_kappas = kappa_metrics.val_kappas

epochs_range = range(len(history.history["loss"]))

plt.figure(figsize = (16, 16))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label = 'Training Accuracy')
plt.plot(epochs_range, val_acc, label = 'Validation Accuracy')
plt.plot(epochs_range, val_kappas, label = 'Validation Kappa')
plt.legend(loc = 'lower right')
plt.title('Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label = 'Training Loss')
plt.plot(epochs_range, val_loss, label = 'Validation Loss')
plt.legend(loc = 'upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
print(y_pred)

In [ ]:
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
print(y_pred)

In [ ]:
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, predictions, target_names=["C 0", "C 1", "C 2", "C 3", "C 4"]))

> if time permits:
* test on external data? 
* add data from resized 2015 for the classes with low accuracy and retrain/test    

# Fast Gradient Attack


In [ ]:
from tensorflow.keras.losses import BinaryCrossentropy
loss_object = BinaryCrossentropy()
import tensorflow as tf
def generate_image_adversary(model, image, label, eps=2 / 255.0):
    image = tf.cast(image, tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(image)
        pred = model(image)
       # print(label, pred)
        #label = label.astype(int).sum(axis=0) - 1
        #pred = pred.astype(int).sum(axis=0) - 1
        label = tf.convert_to_tensor(label)
        label = tf.reshape(label, (1, 5))
        
        #print(label, pred)
        loss = loss_object(label, pred)
        gradient = tape.gradient(loss, image)
        signedGrad = tf.sign(gradient)
        adversary = (image + (signedGrad * eps)).numpy()
    return adversary

In [ ]:
import numpy as np
def generate_adversarial_batch(model, total, images, labels, dims,eps=0.01):
    (h, w, c) = dims
    while True:
        perturbImages = []
        perturbLabels = []
        idxs = np.random.choice(range(0, len(images)), size=total,replace=False)
        for i in idxs:
            image = images[i]
            label = labels[i]
            adversary = generate_image_adversary(model,image.reshape(1, h, w, c), label, eps=eps)
            perturbImages.append(adversary.reshape(h, w, c))
            perturbLabels.append(label)
        yield (np.array(perturbImages), np.array(perturbLabels))

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Normal Accuracy : {acc}")

In [ ]:
print("generating adversarial examples with FGSM...")
(advX, advY) = next(generate_adversarial_batch(model, len(X_val),X_val, Y_val, (256,256, 3), eps=0.01))
# re-evaluate the model on the adversarial images


In [ ]:
y_pred = model.predict(advX) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =advY.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Adv Accuracy : {acc}")

In [ ]:
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
print("fine-tuning network on adversarial examples...")
model.fit(advX, advY,batch_size=16,epochs=5,verbose=1)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(advX[0], interpolation='nearest')
plt.title("Adversial Image of " + get_category(int(advY[1].sum(axis=0)) - 1))
plt.show()

In [ ]:
y_pred = model.predict(X_val) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =Y_val.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Normal Accuracy after fine-tuning: {acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")

In [ ]:
y_pred = model.predict(advX) > 0.5#get the probabilities of multiclasses
y_pred = y_pred.astype(int).sum(axis=1) - 1#retrieve index 0 based
predictions = y_pred.reshape(1,-1)[0]
y_val =advY.astype(int).sum(axis=1) - 1#retrieve index 0 based
acc = accuracy_score(y_val, predictions)
print(f"Adv Accuracy after fine-tuning : {acc}")

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, predictions)
import seaborn as sns
sns.heatmap(cm, annot = True, cmap = "Blues")